In [2]:
# %matplotlib qt 

# %matplotlib inline

import typing as T
import numpy.typing as npt

import numpy as np
import scipy as sp

from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator

from pydrake.solvers import (  
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
)
from pydrake.math import le, eq
from pydrake.symbolic import Polynomial, Variable, Variables

from util import timeit

In [3]:
%matplotlib widget

In [4]:
class Vertex:
    def __init__(self, name=""):
        self.bounds = dict()
        self.name = name
        self.edges_in = [] 
        self.edges_out = []
        
    def add_edge_in(self, edge):
        self.edges_in.append(edge)

    def add_edge_out(self, edge):
        self.edges_out.append(edge)

    def add_measure_conservation_constraints(self, prog:MathematicalProgram, start_measure = None, target_measure = None):
        if len(self.edges_out) > 0:
            measure_out = sum([ e.get_left_measure() for e in self.edges_out])
        else:
            assert target_measure is not None
            measure_out = target_measure

        if len(self.edges_in) > 0:
            measure_in = sum([ e.get_right_measure() for e in self.edges_in ])
        else:
            assert start_measure is not None
            measure_in = start_measure

        prog.AddLinearConstraint(eq(measure_in, measure_out))

    def get_measure(self, solution:MathematicalProgramResult):    
        left_m = sum([ e.get_right_measure(solution) for e in self.edges_in ])
        right_m = sum([ e.get_left_measure(solution) for e in self.edges_out ])
        if len(self.edges_in) == 0:
            return np.round(right_m,3)
        elif len(self.edges_out) == 0:
            return np.round(left_m,3)
        else:
            assert np.allclose(left_m, right_m, 1e-3)
            return np.round(left_m,3)




class BoxVertex(Vertex):
    # box, 2nd degree
    def __init__(self, name, x_min:float, x_max:float, y_min:float, y_max:float):
        super(BoxVertex, self).__init__(name)
        self.bounds = {"x_min": x_min, "x_max":x_max, "y_min":y_min, "y_max":y_max}

    def get_quadratic_constraints(self):
        # return two quadratic constraints:
        # (x-min)(max-x) = -x^2 +(max+min)x - min*max
        x_min, x_max = self.bounds["x_min"], self.bounds["x_max"] 
        y_min, y_max = self.bounds["y_min"], self.bounds["y_max"]
        # return coefficients that will go with mu^2, mu, 1 -- for both x and y
        return np.array([ [-1, (x_max+x_min), -x_max*x_min], [-1, (y_max+y_min), -y_max*y_min] ])
    
    def get_singleton_measure(self,x=None, y=None):
        x_min, x_max = self.bounds["x_min"], self.bounds["x_max"] 
        y_min, y_max = self.bounds["y_min"], self.bounds["y_max"]
        if x is not None and y is not None:
            assert x_min <= x <= x_max
            assert y_min <= y <= y_max
        else:
            x,y = (x_min+x_max)/2, (y_min+y_max)/2
        return np.array( [[1,x,y],[x,x**2, x*y],[y, x*y, y**2]] )

class PointVertex(Vertex):
    # point, 2nd degree
    def __init__(self, name, x:float, y:float):
        super(PointVertex, self).__init__(name)
        self.bounds = {"x": x, "y":y}

    def get_singleton_measure(self):
        x,y = self.bounds["x"], self.bounds["y"]
        return np.array( [[1,x,y],[x,x**2, x*y],[y, x*y, y**2]] )

class Edge:
    def __init__(self, v_left: Vertex, v_right: Vertex, prog: MathematicalProgram):
        self.left = v_left.name
        self.right = v_right.name

        self.define_psd_distribution_matrix(prog)
        self.add_quadratic_cost(prog)
        self.add_constraints(prog, v_left, v_right)
        v_left.add_edge_out(self)
        v_right.add_edge_in(self)

    def define_psd_distribution_matrix(self, prog: MathematicalProgram):
        self.edge_measure = prog.NewSymmetricContinuousVariables(rows=5)
        prog.AddPositiveSemidefiniteConstraint(self.edge_measure)
        # 1 x    y     u     v
        # x x^2  xy    ux    vx
        # y xy   y^2   uy    vy
        # u xu  uy     u^2   uv
        # v xv  vy     uv    v^2

    def get_left_measure(self, solution:MathematicalProgramResult=None):
        m = self.edge_measure[:3, :3]
        if solution is None:
            return m
        else:
            return solution.GetSolution(m)

    def get_right_measure(self, solution:MathematicalProgramResult = None):
        m = np.vstack( (np.hstack( (self.edge_measure[0,0], self.edge_measure[0, 3:]) ),
                    np.hstack( (self.edge_measure[3:,0:1], self.edge_measure[3:, 3:]) ) ))
        if solution is None:
            return m
        else:
            return solution.GetSolution(m)

    def add_quadratic_cost(self, prog: MathematicalProgram):
        C = np.array(
            [[0,0,0,0,0],
             [0,1,0,-1,0],
             [0,0,1,0,-1],
             [0,-1,0,1,0],
             [0,0,-1,0,1]]
        )
        prog.AddLinearCost(np.trace(C @ self.edge_measure))

    def add_constraints(self, prog:MathematicalProgram, left:Vertex, right:Vertex):
        mu_1 = self.edge_measure[0,0]
        if type(left) == BoxVertex:
            self.add_box_constraints_to_measure( prog, self.get_left_measure(), left.get_quadratic_constraints() )
        elif type(left) == PointVertex:
            prog.AddLinearConstraint( eq( self.get_left_measure(), mu_1*left.get_singleton_measure() ) )

        if type(right) == BoxVertex:
            self.add_box_constraints_to_measure( prog, self.get_right_measure(), right.get_quadratic_constraints() )
        elif type(right) == PointVertex:
            prog.AddLinearConstraint( eq( self.get_right_measure(), mu_1*right.get_singleton_measure() ) )

    def add_box_constraints_to_measure(self, prog:MathematicalProgram, measure, constraints):
        mu_1 = measure[0,0]
        mu_x = measure[0,1]
        mu_y = measure[0,2]
        mu_x2 = measure[1,1]
        mu_y2 = measure[2,2]

        mu_x2_x_1 = np.array([ mu_x2, mu_x, mu_1 ])
        mu_y2_y_1 = np.array([ mu_y2, mu_y, mu_1 ])

        prog.AddLinearConstraint( constraints[0].dot( mu_x2_x_1 ) >= 0 )
        prog.AddLinearConstraint( constraints[1].dot( mu_y2_y_1 ) >= 0 )

In [10]:
prog = MathematicalProgram()

vt = PointVertex("t", 0, -0.5)
v1 = BoxVertex("v1", -3,3, 1,2)
v2 = BoxVertex("v2", -4,-2, 3,4)
v3 = BoxVertex("v3", 2,4, 3,4)
v4 = BoxVertex("v4", -3,3, 5,6)
v5 = BoxVertex("v5", -3,3, 7,8)
vs = BoxVertex("s", -3,3,9,10)

vertices = [vt,v1,v2,v3,v4,vs]

Edge(v1, vt, prog)
Edge(v2, v1, prog)
Edge(v3, v1, prog)
Edge(v4, v2, prog)
Edge(v4, v3, prog)
Edge(v5, v4, prog)
Edge(vs, v5, prog)



vs.add_measure_conservation_constraints(prog, start_measure = vs.get_singleton_measure(0., 9.5))
v1.add_measure_conservation_constraints(prog)
v2.add_measure_conservation_constraints(prog)
v3.add_measure_conservation_constraints(prog)
v4.add_measure_conservation_constraints(prog)
v5.add_measure_conservation_constraints(prog)
vt.add_measure_conservation_constraints(prog, target_measure= vt.get_singleton_measure())

solution = Solve(prog)
print(solution.is_success())
print(solution.get_optimal_cost())

True
23.33330178724262


In [11]:
for v in vertices:
    print("------" + v.name)
    print(v.get_measure(solution))

------t
[[ 1.    0.   -0.5 ]
 [ 0.    0.   -0.  ]
 [-0.5  -0.    0.25]]
------v1
[[1.   0.   1.5 ]
 [0.   1.   0.  ]
 [1.5  0.   2.25]]
------v2
[[ 0.5   -1.     1.75 ]
 [-1.     2.    -3.5  ]
 [ 1.75  -3.5    6.125]]
------v3
[[0.5   1.    1.75 ]
 [1.    2.    3.5  ]
 [1.75  3.5   6.125]]
------v4
[[ 1.     0.     5.5  ]
 [ 0.     1.778  0.   ]
 [ 5.5    0.    30.25 ]]
------s
[[ 1.   -0.    9.5 ]
 [-0.    0.    0.  ]
 [ 9.5   0.   90.25]]


In [125]:
prog = MathematicalProgram()

vs = BoxVertex("s", -3,3,7,9)
v1 = BoxVertex("v1", -3,3, 1,2)
v2 = BoxVertex("v2", -3,3, 3,4)
v3 = BoxVertex("v3", -4,-2, 5,6)
v4 = BoxVertex("v4", 2,4, 5,6)
vt = PointVertex("t", 0,0)

vertices = [vt,v1,v2,v3,v4,vs]

Edge(v1, vt, prog)
Edge(v2, v1, prog)
Edge(v3, v2, prog)
Edge(v4, v2, prog)
Edge(vs, v4, prog)
Edge(vs, v3, prog)


# vs.add_measure_conservation_constraints(prog, start_measure = vs.get_singleton_measure(0, 8))
vs.add_measure_conservation_constraints(prog, start_measure = vs.get_singleton_measure(-1, 8)+vs.get_singleton_measure(1, 8) )
v1.add_measure_conservation_constraints(prog)
v2.add_measure_conservation_constraints(prog)
v3.add_measure_conservation_constraints(prog)
v4.add_measure_conservation_constraints(prog)
vt.add_measure_conservation_constraints(prog, target_measure= 2*vt.get_singleton_measure())

solution = Solve(prog)
print(solution.is_success())
print(solution.get_optimal_cost())

True
36.6666360166778


In [126]:
for v in vertices:
    print("------" + v.name)
    print(v.get_measure(solution))

------t
[[ 2.  0. -0.]
 [ 0.  0. -0.]
 [-0. -0.  0.]]
------v1
[[ 2.     0.     3.999]
 [ 0.     0.889 -0.   ]
 [ 3.999 -0.     7.998]]
------v2
[[ 2.     0.     7.999]
 [ 0.     3.556 -0.   ]
 [ 7.999 -0.    31.994]]
------v3
[[  1.     -2.      6.   ]
 [ -2.      4.    -11.999]
 [  6.    -11.999  35.996]]
------v4
[[ 1.     2.     6.   ]
 [ 2.     4.    11.999]
 [ 6.    11.999 35.996]]
------s
[[  2.  -0.  16.]
 [ -0.   2.   0.]
 [ 16.   0. 128.]]


In [ ]:
# ------t
# [[ 1.  0. -0.]
#  [ 0.  0. -0.]
#  [-0. -0.  0.]]
# ------v1
# [[ 1.    -0.     2.   ]
#  [-0.     0.444 -0.   ]
#  [ 2.    -0.     3.999]]
# ------v2
# [[ 1.0000e+00 -0.0000e+00  4.0000e+00]
#  [-0.0000e+00  1.7780e+00 -1.0000e-03]
#  [ 4.0000e+00 -1.0000e-03  1.5997e+01]]
# ------v3
# [[ 0.5   -1.     3.   ]
#  [-1.     2.    -6.001]
#  [ 3.    -6.001 18.001]]
# ------v4
# [[ 0.5    1.     2.999]
#  [ 1.     2.     5.999]
#  [ 2.999  5.999 17.995]]
# ------s
# [[ 1. -0.  8.]
#  [-0.  0. -0.]
#  [ 8. -0. 64.]]